In [ ]:
import resa_uv_abs as uv

# Upload UV absorbance data to RESA2

This notebook reads UV absorbance data produced by NIVA Lab's spectrophotometer and adds it to RESA2. It provides an alternative to Tore's old Access database named `ABSDATA_IMPORT.accdb`. This is a temporary solution, but will hopefully be faster/more efficient than the current workflow.

#### Description of workflow

 * The analyses are conducted in batches by Erling Bratsberg. Result files for each batch are stored in folders named `AB{yymmdd}` where `{yymmdd}` is the day the analyses were conducted. The spectrophotometer write data automatically to the Lab's network drive at `T:\LAMBDA40P\UVWINLAB\DATA`, and Erling also copies them to `K:\Avdeling\412 Ana\LAMBDA\ABSSPEKTER_KAU` so that Liv Bente can access them. **Note:** The directory structure in these locations is broadly - but not exactly - the same <br><br>
 
 * Within each folder, result files are named `xxxxx.SP` where `xxxxx` is the serial number used in Labware. Each sample can be uniquely identified in Labware by the Labware text ID, which is formed as `NR-{year}-{serial_no}`. Each file contains a header, plus UV absorbance results for 701 wavelengths from 200 to 900 nm inclusive <br><br>
 
 * Each folder should also contain one or more "blank"/calibration files, usually named either `BLANK.SP` or `BL.SP`. Sometimes both files are present. An analysis will begin with a blank file (usually called `BLANK.SP`) followed by several result files, then another blank file (usually `BL.SP`) and some more samples. The first blank file applies to the first set of results and the second to the second set of results. All the files include date & time stamps in the header, which can be used to match result files to blanks <br><br>
 
 * The folders may contain files with other extensions, but these can be ignored for the purposes of this workflow <br><br>
 
 * The spectrophotometer uses cuvettes with a fixed length of 5 cm. UV absorbance is usually reported per cm, so values should be divided by 5 before upload <br><br>
 
 * Sometimes, dark samples must be diluted before analysis. In these cases, a dilution factor should also be reported. **Need to find out where this is stored and include it in the code** <br><br>

Corrected values are calculated as
 
$$A_{cor} = \frac{D(A_{raw} - A_{blank})}{L}$$

where $A_{cor}$ is the corrected value to be uploaded; $A_{raw}$ and $A_{blank}$ are the raw and calibration absorbances at each wavelength, respectively; $D$ is the dimensionless dilution factor; and $L$ is the cuvette length in cm.

## 1. User options

Take care with setting `force_update = True` in the code below. This will delete existing data from the database and upload it again.

#### Issues to solve

 * The dilution factor should eventually be identified automatically in the code. I have a placeholder function for this, named `get_dilution`, but it current always returns the value 1. **This must be changed before the code is used "for real"**

In [ ]:
# Data folder to process
uv_data_fold = r"../../test_data"

# Whether to re-upload data for samples already in the database.
# If 'force_update' is False and UV data for the current water sample are
# already present in RESA2, data for this sample will be skipped (and a
# message printed to the output). If 'force_update' is True, data already
# in the database for the current sample will be deleted and the values
# uploaded again
force_update = False

In [ ]:
# Check user input
assert type(force_update) is bool, "'force_update' must be of Boolean type."

## 2. Upload data

**You must login as a user with sufficient permissions to write to tables in RESA2**.

The code performs the following steps:

 1. Gets a list of all folders within `uv_data_fold` that begin with the letters `AB` (capitalised) <br><br>
 
 2. For each folder, gets a list of all files ending with extension `.SP` and identifies the blank files (which are assumed to be named `BL*.SP`) <br><br>
 
 3. Assigns a blank file to each result file based on analysis date and time (see e-mail from Liv Bente received 19.10.2021 at 13.48) <br><br>
 
 4. For each (non-blank) `.SP` file in each folder, identifies the serial number from the file name. Also identifies the year when the analysis was conducted and constructs the Labware text ID as `NR-{year}-{serial_no}`. This information is used to identify the corresponding RESA2 water sample ID. The code raises an error if more than one sample ID is found, and prints a warning (but continues) if no matching ID can be identified <br><br>
 
 5. Reads each raw data file and the corresponding blank, and checks both contain data for 701 wavelengths <br><br>
 
 6. The blank values are subtracted from the raw values, and the result corrected for dilution and cuvette length <br><br>
 
 7. Corrected values are uploaded to `RESA2.ABSORBANCE_SPECTRAS`. If this table already contains values for the water sample being processed, a warning will be printed and the code will continue to the next file without uploading anything (unless `force_update = True`, in which case the existing data will be deleted from RESA and uploaded again) <br><br>
 
 8. Within each folder, a subfolder is created named `uploaded`. Each raw `.SP` file that is successfully processed is moved to this subfolder <br><br>
 
 9. When a sample has been successfully processed, a new row is inserted into `RESA2.LOG_ABS_SPECTRA`. This records key information about the sample (`labware_text_id`, `water_sample_id`, `year`, `serial_no`, `blank_file`, `dilution`, `cuvette_len_cm`, `original_path` and `archive_path`), plus the user that uploaded the data and the current date. Note that if a file is uploaded multiple times, it will appear several times in this table (but the dataset can only appear once in `RESA2.ABSORBANCE_SPECTRAS`, as it must be deleted each time before uploading again) <br><br>
 
 10. Progress (and error) messages are printed to the console and also logged in a file named `resa2_uvabs_log_{log_date}.txt` <br><br>
 
 11. Once all folders within `uv_data_fold` have been traversed, the code sends an e-mail from `resa2.uvabs@gmail.com` with the log file as an attachment

In [ ]:
# Process data
uv.main(uv_data_fold=uv_data_fold, force_update=force_update)